# TODO:

1. IV APPROACH
2. PART 4 PLOTTING
3. PART 4 EFFECT REMOVAL

## Macro 2: Problem Sheet 1

### Exercise 1

> **Remark 1.** This notebook is structured as follows. First I define all necessary functions in a sort of abstract fashion. Then at the very end I call all functions to produce numerical results and plots.

> **Remark 2.** Either our data management contains errors or the data is not suitable for our analysis since many estimation steps need a subset of the data that is empty. See section [*Remark on the data*](#remark-on-the-data).

In [1]:
from pathlib import Path
from functools import partial

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.formula.api import ols
from joblib import Parallel
from joblib import delayed

In [2]:
DATA_PATH = Path("data/CNEF_PSID")

### Functions

In [3]:
def compose(f, g):
    """Composition of two functins.
    
    h(x) := f(g(x)) for all x
    
    """
    h = lambda x: f(g(x))
    return h

In [4]:
def get_yearly_variables(year):
    """Returns list and dict of relevant columns given year.
    
    Convert variables names to year specific names. For example instead
    of 'x11102' writes 'x1110285' for the year '85. Additionally add
    variable name for individual id.
    
    For details on variable names and more information seek the
    codebook: https://www.cnefdata.org/documentation/codebooks
    
    
    Args:
        year (int): Year. (Write 85 for 1985.)
        
    Returns:
        as_list (list): List of variable names.
        as_dict (dict): Dict of variable name translation to human
            readable form.
    
    """
    variables = {
        "x11102": "household",
        "i11102": "income",
        "d11105": "relationship_to_head",
        "d11101": "age",
        "d11109": "education",
        "e11101": "hours",
    }
    as_dict = {f"{key}{year}": value for key, value in variables.items()}
    as_dict = dict(as_dict, **{"x11101ll": "individual"})
    as_list = [f"{key}{year}" for key in variables.keys()]
    as_list.append("x11101ll")
    return as_dict, as_list

In [5]:
def load_data_given_year(year):
    """Load data and assign new columns given year.
    
    This already does some steps described in part 2 of exercise 1.
    
    Args:
        year (int): Year. (Write 85 for 1985.)
        
    Returns:
        df (pd.DataFrame): Data frame with columss
    
    """
    cols_mapper, cols = get_yearly_variables(year)

    df = pd.read_stata(DATA_PATH / f"pequiv{year}.dta", columns=cols)

    df = df.rename(columns=cols_mapper)
    df = df.set_index(["household", "individual"])
    df = df.dropna(how="all")
    df = df.assign(
        **{
            "year": year,
            "relationship_to_head": df.relationship_to_head.str.split(" ").apply(
                lambda s: s[0]
            ),
        }
    )
    df = df.convert_dtypes()
    return df

In [6]:
def clean_data(df):
    """Clean data frame.
    
    This does most steps described in part 2 of exercise 1.
    
    Args:
        df (pd.DataFrame): Frame produced by :func:`load_data_given_year`.
        
    Returns:
        df (pd.DataFrame): Cleaned data frame.
    
    """
    TEN_PERCENT_OF_ANNUAL_FULLTIME_HOURS = 208

    df = df.query("relationship_to_head in ['head', 'partner']")
    df = df.assign(
        **{
            "is_single": df.relationship_to_head.groupby("household").transform(
                lambda x: set(x) != {"head", "partner"}
            )
        }
    )
    df = df.query("is_single == False")

    total = df.groupby(by="household")[["hours", "income"]].sum()

    df = df.query("relationship_to_head == 'head'")
    df = df.reset_index(level="individual")
    df = df.assign(**{"income": total.income, "hours": total.hours})

    df = df.query("25 <= age < 56")
    df = df.query("hours >= @TEN_PERCENT_OF_ANNUAL_FULLTIME_HOURS")
    df = df.query("income > 0")

    df = df.assign(**{"income": np.log(df.income)})

    df = df.drop(["relationship_to_head", "is_single", "hours"], axis=1)
    df = df.astype(
        {
            "income": float,
            "education": "category",
            "age": "category",
            "year": "category",
        }
    )
    df = df.set_index("age", append=True)
    df = df.dropna(how="any")
    return df

In [7]:
def load_and_clean_data(n_jobs=1, load_from_disc=False):
    """Load, clean and merge data for years 1980 to 1997.
    
    Since loading and cleaning the data is time consuming there is
    a check if the clean data is already available.
    
    Args:
        n_jobs (int): Number of cores to use for parallelized data cleaning.
        load_from_disc (bool): Should the dataset be loaded from disc if available.
            Default False.
    
    Returns:
        df (pd.DataFrame): Cleaned and merged data frame with index ['household', 'year']
            and columns 'income', 'age', 'education' and 'work'. Column 'income' is float
            while all other columns are category.
    
    """
    clean_data_path = DATA_PATH / "clean_data.csv"
    if load_from_disc and clean_data_path.exists():
        df = pd.read_csv(clean_data_path)
    else:
        _load_and_clean = compose(clean_data, load_data_given_year)
        dfs = Parallel(n_jobs, prefer="processes")(
            delayed(_load_and_clean)(year) for year in range(80, 98)
        )
        df = pd.concat(dfs).sort_index().reset_index()
        
    df = df.drop_duplicates(subset=["household", "age"], keep="first")
    return df

In [8]:
def fit_dummy_regression(df):
    """Fit dummy regression on data in df.
    
    In the formula object C() tells statsmodels to use the variable as
    categorical variable.
    
    """
    model = ols("income ~ C(year) + C(age) + C(education)", data=df)
    model = model.fit()
    return model

In [9]:
def estimate_quantities_approach_4(df):
    """Estimate quantities using approach in part 4.
    
    Args:
        df (pd.DataFrame): Cleaned data with residuals.
    
    Returns:
        quantities (pd.DataFrame): Estimates of rho, sigma_eps^2 and sigma_mu_tau^2.
        var (pd.DataFrame): Sample variances.

    """
    var = df.query("age in [25, 40, 55]")[["age", "residuals"]].groupby("age").var()

    _rho = (var.loc[55][0] - var.loc[40][0]) / (var.loc[40][0] - var.loc[25][0])
    rho = _rho ** (1 / 30)
    gamma = rho ** 2 * (1 - rho ** 30) / (1 - rho ** 2)

    sigma_eps = (var.loc[40][0] - var.loc[25][0]) / gamma
    sigma_mu_tau = var.loc[25][0] - sigma_eps

    quantities = pd.DataFrame(
        [rho, sigma_eps, sigma_mu_tau],
        columns=["value"],
        index=["$\rho$", "$\sigma_{\epsilon}^2$", "$\sigma_{\mu\tau}^2$"],
    )
    return quantities, var

In [10]:
def estimate_quantities_approach_5(df, var):
    """Estimate quantities using approach in part 5.
    
    Args:
        df (pd.DataFrame): Cleaned data with residuals.
    
    """
    forty_year_old_households = df.query("age == 40")["household"]
    df = df.query("household in @forty_year_old_households")

    # compute (sample) covariances
    combinations = [(40, 39), (40, 38), (40, 37)]
    cov = pd.DataFrame(index=pd.MultiIndex.from_tuples(combinations))
    for comb in combinations:

        _df = df[["household", "age", "residuals"]].query("age in @comb")
        idx = _df.groupby("household")["age"].transform(
            lambda x: set(x) == set(comb) and len(x) == 2
        )
        _df = _df.loc[idx,].set_index(["household", "age"])

        _cov = _df.unstack(level="age").cov().values[0, 1]
        cov.loc[comb, "cov"] = _cov

    # compute quantities
    rho = (
        (cov.loc[(40, 37),] - cov.loc[(40, 38)])
        / (cov.loc[(40, 38),] - cov.loc[(40, 39)])
    )[0]

    sigma_eps = (
        (cov.loc[(40, 37),] - cov.loc[(40, 37),])[0]
        * (1 - rho ** 2)
        / (rho * (rho - 1) * (rho ** 29 + 1))
    )
    sigma_mu = cov.loc[(40, 39),][0] - sigma_eps * rho * (1 - rho ** 30) / (
        1 - rho ** 2
    )
    sigma_tau = var.loc[25][0] - sigma_mu - sigma_eps

    quantities = pd.DataFrame(
        [rho, sigma_eps, sigma_mu, sigma_tau],
        columns=["value"],
        index=[
            "$\rho$",
            "$\sigma_{\epsilon}^2$",
            "$\sigma_{\mu}^2$",
            "$\sigma_{\tau}^2$",
        ],
    )
    return quantities

In [11]:
def estimate_rho_approach_iv(df, n_jobs=1):
    """Estimate rho using an instrumental variable approach.
    
    Regression equation:
        
        (1)   x_{ih}* = rho x_{i, h-1}* + error
        
    Equation (1) has an endogeneity problem, hence we will construct
    a valid instrument z_{i, h-1} for x_{i, h-1}* and estimate rho
    using 2SLS.
    
    (!) A minimum requirement for this to work is that for *some* number
    of observations x_{ih}* AND x_{i, h-1}* is observed.
    
    Args:
        df (pd.DataFrame): Cleaned data with residuals.
        n_jobs (int): Number of cores to use for parallelized data cleaning.
        
    Returns:
    
        ... MISSING ...
    
    """
    def _create_instrument_or_return_nan(household, age, df):
        try:
            instrument = df.loc[(household, age - 1), "residual"] - df.loc[(household, age - 2), "residual"]
        except KeyError:
            instrument = np.nan
            
        try:
            lagged_residual = df.loc[(household, age - 1), "residual"]
        except KeyError:
            lagged_residual = np.nan
            
        return instrument, lagged_residual
    
    df = df.set_index(["household", "age"])
    
    to_parallelize = partial(_create_instrument_or_return_nan, **{"df": df})
    results = Parallel(n_jobs, prefer="processes")(
        delayed(to_parallelize)(household, age) for household, age in df.index
    )
    results = pd.DataFrame(results, columns=["instrument", "lagged_residuals"], index=df.index)
    
    df = pd.concat((df, results), axis=1)

    # from statsmodels.formula.api import mixedlm
    # https://www.statsmodels.org/stable/mixed_linear.html
    # model = mixedlm("residual ~ lagged_residual", data=df, groups=df.index.get_level_values("household"))
    # check out: https://bashtage.github.io/linearmodels/doc/index.html
    
    quantity = pd.Series([np.nan], index=["$\rho$"], name="value").to_frame()
    return quantity

In [12]:
def combine_parts(part4, part5, part6):
    """Combine results from part4 to part6 in one data frame."""
    keys = ["part4", "part5", "part6"]
    df = pd.concat((part4, part5, part6), axis=1, keys=keys)
    df = df.droplevel(level=1, axis=1)
    df = df.convert_dtypes()
    return df

### Remark on the Data

Before we continue with the actual computation let me make some remarks about the data and why we will expect non-robust estimates from it. All complaints about the data rely on the fact that the data cleaning step are correct. 

In [13]:
df = load_and_clean_data(n_jobs=4)

> **Remark 3.** Apparently the individual who is reported to be household head can change over the years. This causes complications like the following, where for a single household head the age and year columns don't quite match. Furthermore as this example already shows, the age column of an observation is very messy and we do not have many consecutive observations. This will later cause problems for the covariance estimation (part 5) and for the instrument creation (part 6) ---see remark 4.

In [14]:
df.query("household == 1")

,household,age,individual,income,education,year
0,1,35,1397004.0,11.306651,17,91
1,1,41,5065172.0,10.562056,11,83
3,1,54,733001.0,10.888048,13,82


### Fit Model and use Residuals

In [15]:
model = fit_dummy_regression(df)

In [16]:
df = df.assign(**{"residuals": model.resid})

### Results for Part 4, 5 and 6

In [17]:
part4, var = estimate_quantities_approach_4(df)

In [18]:
part5 = estimate_quantities_approach_5(df, var)

In [19]:
part6 = estimate_rho_approach_iv(df, n_jobs=4)

In [20]:
result = combine_parts(part4, part5, part6)

In [21]:
result

,part4,part5,part6
$\rho$,1.010213,1.212954,<NA>
$\sigma_{\epsilon}^2$,0.002043,-0.0,<NA>
$\sigma_{\mu\tau}^2$,0.19632,<NA>,<NA>
$\sigma_{\mu}^2$,<NA>,0.01092,<NA>
$\sigma_{\tau}^2$,<NA>,0.187443,<NA>
